### SETUP

In [ ]:
!sudo apt-get update
!sudo apt-get install nasm
!sudo apt-get install gcc g++
!sudo apt-get install g++-multilib
!!sudo apt-get install gcc-multilib

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 128 kB in 1s (90.7 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it 

['',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 4%',
 '',
 'Reading package lists... 4%',
 '',
 'Reading package lists... 38%',
 '',
 'Reading package lists... 41%',
 '',
 'Reading package lists... 41%',
 '',
 'Reading package lists... 42%',
 '',
 'Reading package lists... 42%',
 '',
 'Reading package lists... 48%',
 '',
 'Reading package lists... 48%',
 '',
 'Reading package lists... 56%',
 '',
 'Reading package lists... 56%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 66%',
 '',
 'Reading package lists... 66%',
 '',
 'Reading pack

In [ ]:
!sudo apt-get install -f

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 58 not upgraded.


In [ ]:
!nasm -v
!g++ -v

NASM version 2.15.05
Using built-in specs.
COLLECT_GCC=g++
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/11/lto-wrapper
OFFLOAD_TARGET_NAMES=nvptx-none:amdgcn-amdhsa
OFFLOAD_TARGET_DEFAULT=1
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu 11.4.0-1ubuntu1~22.04' --with-bugurl=file:///usr/share/doc/gcc-11/README.Bugs --enable-languages=c,ada,c++,go,brig,d,fortran,objc,obj-c++,m2 --prefix=/usr --with-gcc-major-version-only --program-suffix=-11 --program-prefix=x86_64-linux-gnu- --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --libdir=/usr/lib --enable-nls --enable-bootstrap --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --with-default-libstdcxx-abi=new --enable-gnu-unique-object --disable-vtable-verify --enable-plugin --enable-default-pie --with-system-zlib --enable-libphobos-checking=release --with-target-system-zlib=auto --enable-objc-gc=auto --enable-m

### P 1

In [ ]:
%%writefile hello.asm

section .text
    global write_message             ; Make the function globally accessible

write_message:
    ; Save registers that will be modified
    push ebx
    push ecx
    push edx

    ; Get the pointer to the message
    mov ecx, [esp+16]  ; pointer to the message (passed from C)

print_loop:
    mov al, [ecx]      ; load the byte at ecx into al
    cmp al, 0          ; compare the byte with 0
    je  print_done     ; if zero, end of string

    ; sys_write for one character
    mov eax, 4         ; sys_write system call number
    mov ebx, 1         ; file descriptor (stdout)
    mov edx, 1         ; length is 1
    int 0x80           ; make the system call

    inc ecx            ; move to the next character
    jmp print_loop     ; repeat the loop

print_done:
    ; Restore registers
    pop edx
    pop ecx
    pop ebx

    ret                ; return to the caller

Writing hello.asm


In [ ]:
%%writefile main.c

#include <stdio.h>

// Declare the assembly function
extern void write_message(const char* message);

int main() {
    const char* message = "Hello, World!\n";

    write_message(message);  // Call the assembly function with the string
    printf("Returning to C code.\n");

    return 0;
}

Writing main.c


In [ ]:
!nasm -f elf32 hello.asm -o hello.o
!gcc -m32 main.c hello.o -o main

In [ ]:
!./main

Hello, World!
Returning to C code.


### P 2

In [ ]:
%%writefile add.asm

; add_func.asm
SECTION .text
    global add_func

add_func:
    ; Save registers that will be modified
    push ebx
    push ecx
    push edx

    ; Save registers that will be modified
    push ebp
    mov ebp, esp

    ; Return the result
    mov eax, [ebp+20]
    add eax, [ebp+24]

    ; Restore the base pointer and return
    mov esp, ebp
    pop ebp

    ; Restore registers
    pop edx
    pop ecx
    pop ebx

    ret

Writing add.asm


In [ ]:
%%writefile main.cpp

#include <stdio.h>

// Declare the assembly function
extern "C" int add_func(int a, int b);

int main() {

    int result = add_func(14, 22);
    printf("Result: %d\n", result);
    return 0;
}

Writing main.cpp


In [ ]:
!nasm -f elf32 add.asm -o add.o
!g++ -m32 -c main.cpp -o main.o
!g++ -m32 main.o add.o -o main

In [ ]:
!./main

Result: 36


### P 3

In [ ]:
%%writefile printf_int.asm

section .text
global print_int

print_int:

    ; Save registers that will be modified
    push eax
    push ebx
    push ecx
    push edx

    mov eax, 4
    mov ebx, 1
    mov ecx, msg
    mov edx, len
    int 0x80

    mov eax, [esp+20]
    mov ecx, 10
    mov ebx, buffer

loop1:
    xor edx, edx
    div ecx
    add dl, 48

    mov [ebx], dl
    inc ebx

    cmp eax, 0
    je reverse

    jmp loop1

reverse:
    ; Reverse the string in buffer
    mov ecx, buffer
    mov edx, buffer
loop3:
    cmp byte [edx], 0
    je reverse2
    inc edx
    jmp loop3

reverse2:
    dec edx
    mov al, [edx]
    mov ah, [ecx]
    mov [edx], ah
    mov [ecx], al
    inc ecx
    cmp ecx, edx
    jl reverse2

print:

    ; Calculate the length of the string in buffer
    mov ecx, buffer
    mov edx, 0
loop2:
    cmp byte [ecx], 0
    je print2
    inc edx
    inc ecx
    jmp loop2

print2:
    mov eax, 4
    mov ebx, 1
    mov ecx, buffer
    int 0x80

    ; Restore registers
    pop edx
    pop ecx
    pop ebx
    pop eax

    ret

section .data
msg db 'The integer value is: '
len equ $ - msg
buffer times 10 db 0

Writing printf_int.asm


In [ ]:
%%writefile main.cpp

extern "C" void print_int(int);

int main() {
    int a = 42345;
    print_int(a);
    return 0;
}

Overwriting main.cpp


In [ ]:
!nasm -f elf32 printf_int.asm -o printf_int.o

In [ ]:
!g++ -m32 -c main.cpp -o main.o

In [ ]:
!g++ -no-pie -m32 printf_int.o main.o -o main

In [ ]:
!./main

The integer value is: 42345

### P 4


In [ ]:
%%writefile printf_func.asm

section .text
    global printf_func             ; Make the function globally accessible

printf_func:

    ; Save registers that will be modified
    push ebx
    push ecx
    push edx

    ; Save the base pointer
    push ebp
    mov ebp, esp

    ; Get the pointer to the message
    add ebp, 20
    mov ecx, [ebp]  ; pointer to the message (passed from C)

print_loop:

    mov al, [ecx]      ; load the byte at ecx into al

    cmp al, 0          ; compare the byte with 0
    je  print_done     ; if zero, end of string

    ; Check for '%'
    cmp al, '%'
    je check_next_char

    ; Check for '%'
    cmp al, '\'
    je backslah

    ; sys_write for one character
    mov eax, 4         ; sys_write system call number
    mov ebx, 1         ; file descriptor (stdout)
    mov edx, 1         ; length is 1
    int 0x80           ; make the system call

    inc ecx            ; move to the next character
    jmp print_loop     ; repeat the loop

backslah:

    inc ecx            ; move to the next character

    push ecx
    push eax

    mov eax, 4         ; sys_write system call number
    mov ebx, 1         ; file descriptor (stdout)
    mov ecx, nl        ; Next Line \n
    mov edx, 1         ; length is 1
    int 0x80           ; make the system call

    pop ecx
    inc ecx

    jmp print_loop

check_next_char:

    inc ecx            ; move to the next character
    mov al, [ecx]

    cmp al, 'd'        ; check if the format specifier is 'd'
    je print_int       ; if 'd', handle integer

    cmp al, 's'        ; check if the format specifier is 's'
    je print_str       ; if 's', handle string

    ; unknown format specifier, ignore it
    jmp print_loop

print_int:

    push ecx
    push eax

    ; load the integer from the stack
    add ebp, 4
    mov eax, [ebp]

    ; Convert integer to string
    mov ecx, 10
    mov ebx, buffer

    loop1:
    xor edx, edx
    div ecx
    add dl, 48         ; convert digit to ASCII

    mov [ebx], dl
    inc ebx

    cmp eax, 0
    je reverse         ; if no more digits, reverse string

    jmp loop1

    reverse:
    ; Reverse the string in buffer
    mov ecx, buffer
    mov edx, buffer

    loop3:
    cmp byte [edx], 0
    je reverse2
    inc edx
    jmp loop3

    reverse2:

    dec edx
    mov al, [edx]
    mov ah, [ecx]
    mov [edx], ah
    mov [ecx], al
    inc ecx
    cmp ecx, edx
    jl reverse2

    print:

    ; Calculate the length of the string in buffer
    mov ecx, buffer
    mov edx, 0

    loop2:
    cmp byte [ecx], 0
    je print2
    inc edx
    inc ecx
    jmp loop2

    print2:

    ; sys_write to print the integer string
    mov eax, 4
    mov ebx, 1
    mov ecx, buffer
    int 0x80

    pop eax
    pop ecx

    inc ecx

    jmp print_loop

print_str:

    push ecx

    ; load the pointer to the string from the stack
    add ebp, 4
    mov ecx, [ebp]

    loop_str:

    mov al, [ecx]      ; load the byte at ecx into al

    cmp al, 0          ; compare the byte with 0
    je  str_done       ; if zero, end of string

    ; sys_write for one character
    mov eax, 4         ; sys_write system call number
    mov ebx, 1         ; file descriptor (stdout)
    mov edx, 1         ; length is 1
    int 0x80           ; make the system call

    inc ecx            ; move to the next character
    jmp loop_str       ; repeat the loop

    str_done:

    pop ecx
    inc ecx

    jmp print_loop

print_done:

    ; Restore registers
    pop ebp
    pop edx
    pop ecx
    pop ebx

    ret                ; return to the caller

section .data
    buffer times 10 db 0    ; Buffer for integer conversion
    nl db 0xA               ; message with new line character

Writing printf_func.asm


In [ ]:
%%writefile main.cpp

// main.cpp
#include <stdio.h>

extern "C" void printf_func(const char* format, ...);

int main() {

    const char* str = "Dummy";
    printf_func("%d Just %s String \n%d Just %s String",123,str,98765,str);
    printf("\nEnd Main");

    return 0;
}


Overwriting main.cpp


In [ ]:
!nasm -f elf32 printf_func.asm -o printf_func.o

In [ ]:
!g++ -m32 -c main.cpp -o main.o

In [ ]:
!g++ -no-pie -m32 main.o printf_func.o -o main

In [ ]:
!./main

123 Just Dummy String 
98765 Just Dummy String
End Main

### P 5


In [ ]:
%%writefile main.cpp

#include <stdio.h>
#include <time.h>

// Iterative Factorial
unsigned long long factorial_iterative(int n) {
    unsigned long long result = 1;
    for (int i = 2; i <= n; i++) {
        result *= i;
    }
    return result;
}

int main() {
    int num = 63;
    unsigned long long fact_iterative = factorial_iterative(num);
    printf("Iterative Factorial of %d = %llu\n", num, fact_iterative);
    return 0;
}

Overwriting main.cpp


In [ ]:
%%bash

# Compile the C++ program and generate assembly file
g++ -S -o Iterativemain.asm main.cpp

# Compile the C++ program
g++ -o main main.cpp

# Record the time in nanoseconds before execution
start_time=$(date +%s%N)

# Run the program
./main

# Record the time in nanoseconds after execution
end_time=$(date +%s%N)

# Calculate the duration
duration=$((end_time - start_time))

# Print the time taken in nanoseconds
echo "Time taken: $duration nanoseconds"

Iterative Factorial of 63 = 1585267068834414592
Time taken: 3646972 nanoseconds


In [ ]:
%%writefile main.cpp

#include <stdio.h>
#include <time.h>

// Recursive Factorial
unsigned long long factorial_recursive(int n) {
    if (n == 0 || n == 1) {
        return 1; // Base case
    }
    else {
        return n * factorial_recursive(n - 1); // Recursive call
    }
}

int main() {
    int num = 63;
    unsigned long long fact_recursive = factorial_recursive(num);
    printf("Recursive Factorial of %d = %llu\n", num, fact_recursive);
    return 0;
}

Overwriting main.cpp


In [ ]:
%%bash

# Compile the C++ program and generate assembly file
g++ -S -o Recursivemain.asm main.cpp

# Compile the C++ program
g++ -o main main.cpp

# Record the time in nanoseconds before execution
start_time=$(date +%s%N)

# Run the program
./main

# Record the time in nanoseconds after execution
end_time=$(date +%s%N)

# Calculate the duration
duration=$((end_time - start_time))

# Print the time taken in nanoseconds
echo "Time taken: $duration nanoseconds"

Recursive Factorial of 63 = 1585267068834414592
Time taken: 3584144 nanoseconds


### P 6

In [ ]:
%%writefile str_len.asm

section .text
global str_len

str_len:

    mov edi, [esp+4]  ; Load pointer to string

    mov ecx, -1       ; Set loop counter to -1
    xor al, al        ; Clear al register
    repne scasb       ; Scan string for null terminator
    mov eax, ecx      ; Move negative length to eax

    neg eax           ; Convert to positive length
    dec eax           ; Adjust for null terminator

    ret               ; Return length

Writing str_len.asm


In [ ]:
%%writefile str_cmp.asm

section .text
global str_cmp

str_cmp:
    mov edi, [esp+4]   ; Load pointer to string 1
    mov esi, [esp+8]   ; Load pointer to string 2

    CLD                ; Clear direction flags for string ops

    mov ecx, -1        ; Initialize loop counter to -1

    repz cmpsb         ; Compare bytes in both strings until ECX is 0

    jne found_difference ; If not equal, jump to difference handling

    xor eax, eax       ; If equal, return 0 in EAX
    ret

found_difference:

    ; return difference in char

    ; mov al, [edi-1]   ; Move pointers back to the mismatched characters
    ; mov ah, [esi-1]
    ; sub al, ah        ; Return the difference between the characters
    ; movsx eax, al

    ; Return index of first mismatch
    mov eax, ecx       ; Move loop counter to EAX
    neg eax            ; Convert to positive index
    dec eax            ; Adjust for 0-based indexing
    ret

Writing str_cmp.asm


In [ ]:
%%writefile mem_set.asm

section .text
global mem_set

mem_set:

    CLD                ; clear direction flags

    mov edi, [esp+4]   ; Load array pointer into EDI
    mov ecx, [esp+8]   ; Load array length into ECX

    xor al, al         ; Set AL to 0 (null value)

    rep stosb          ; Fill array with null values

    ret

Writing mem_set.asm


In [ ]:
%%writefile mem_cpy.asm

section .text
global mem_cpy

mem_cpy:

    CLD                ; clear direction flags

    mov edi, [esp+4]   ; Load string pointers into EDI and ESI
    mov esi, [esp+8]
    mov ecx, [esp+12]  ; N bytes to copy

    rep movsb          ; Compare bytes in both strings

    ret


Writing mem_cpy.asm


In [ ]:
%%writefile main.cpp

#include <stdio.h>

// Declare the str_len function, which calculates the length of a string
extern "C" int str_len(const char *str);  // Returns the length of the input string

// Declare the str_cmp function, which compares two strings
extern "C" int str_cmp(const char *str1, const char *str2);  // Returns 0 if equal, non-zero if not equal

// Declare the mem_set function, which sets a block of memory to a specific value
extern "C" void mem_set(const char *destination, int N);  // Sets N bytes of memory at destination to 0

// Declare the mem_cpy function, which copies a block of memory from one location to another
extern "C" void mem_cpy(const char *destination, const char *source, int N);  // Copies N bytes from source to destination

int main() {

    char str[] = "Hello, World! \n";
    int len = str_len(str);
    printf("\nLength of string :: %d \n", len);

    char str1[] = "Hello";
    char str2[] = "Hellz";
    int result = str_cmp(str1, str2);
    printf("\nResult: %d\n", result);

    char source[] = "I am On-line";
    char destination[20];

    mem_set(destination,20);

    printf("\nBefore copying: %s\n", destination);

    mem_cpy(destination, source, 13);

    printf("After copying: %s\n", destination);

    return 0;
}

Overwriting main.cpp


In [ ]:
!nasm -f elf32 mem_cpy.asm -o mem_cpy.o
!nasm -f elf32 mem_set.asm -o mem_set.o
!nasm -f elf32 str_len.asm -o str_len.o
!nasm -f elf32 str_cmp.asm -o str_cmp.o
!gcc -m32 main.cpp mem_cpy.o mem_set.o str_len.o str_cmp.o -o main
!./main


Length of string :: 16 

Result: 5

Before copying: 
After copying: I am On-line
